In [3]:
# pip install langchain langchain_community pypdf PGVector

**Import necessary modules and classes for document loading**

In [240]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores.pgvector import PGVector
from langchain.docstore.document import Document
from db import DB
import os

**Gathering Schema and creating a Document**

In [246]:
job_id = "0fee44ac-bd4b-4217-a64d-a7dc33e00186" 

with open('schema_as_txt.txt', 'r') as file:
    schema_text = file.read().replace("$$ID$$", 'job_id').replace("$$VALUE$$", job_id)

sections = schema_text.strip().split("\n\n")

# Create chunks as Document objects
documents = []
for section in sections:
    documents.append(
        Document(metadata={"source": "schema_as_txt.txt"}, page_content=section.strip())
    )

data = documents

**Define the collection name for storing embeddings and creating vector DB store instance**

In [247]:
def cred(db):
    return PGVector.connection_string_from_db_params(driver='psycopg2',
    user='postgres',
    password='Cymonix.1234',
    host='test-vec-db-instance-1.czmj15zygnmm.us-west-2.rds.amazonaws.com',
    port='5432',
    database= db,
    )

In [248]:
def create_db_inst(database, c_name, data):
# Initialize the text embedding model
    embeddings = BedrockEmbeddings()
    db = database
    CONNECTION_STRING = cred(db)
# Create a vector database store instance and populate it with document data and embeddings
    return PGVector.from_documents(
                                documents=data,
                                embedding=embeddings,
                                collection_name=c_name,
                                connection_string=CONNECTION_STRING
                            )

In [249]:
database = "vec-db"
c_name = "text-schema-2"
data = data 

db1 = create_db_inst(database, c_name, data)

In [250]:
#create or delete database

# DB.create_db(database)
# DB.drop_db(database)

In [251]:
question = "What was the Revenue by Value Stream during the period Q1-2023?"
job_id = "0fee44ac-bd4b-4217-a64d-a7dc33e00186"

In [252]:
retriever= db1.as_retriever(search_type="mmr", search_kwargs={'k': 9, 'lambda_mult': 0.5})
matched_docs = retriever.get_relevant_documents(query=question)
matched_docs

[Document(metadata={'source': 'schema_as_txt.txt'}, page_content='Properties for Node `Value Stream` are:\n$$creationDate$$ as type STRING,\n$$Name$$ as type STRING,\nprogram as type STRING,\napplication as type STRING.'),
 Document(metadata={'source': 'schema_as_txt.txt'}, page_content='Following are the Nodes Label:\n`Sales Partner`,\n`Sales Order`,\n`Work Order`,\n`Customer`,\n`Part`,\n`Value Stream`,\n`Company`,\n`Shipment Provider`,\n`Quote`.'),
 Document(metadata={'source': 'schema_as_txt.txt'}, page_content='Important Note:\nJOB ID:always include job_id = value in WHERE clause,\nVALUE STREAM:value stream should always taken as Node propertie of `Work Order` Node, not from the `Value Stream` Node itself,\nLICENSE PROGRAM:progem should always taken as Node propertie od `Work Order` Node.\nDATE:date should be taken from node `Work Order`,`ship_date`, If required then create a relationship with node `Work Order`, When ever you useing of datetime, use datetime(\'2023-01-01T00:00:00\'

**retrieval chain with the language model and database retriever**

In [342]:
from langchain_community.chat_models import BedrockChat
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import boto3
import json

def create_chain(db, question, job_id):
    # Initialize the Bedrock client using boto3
    client = boto3.client(
        service_name="bedrock-runtime", region_name="us-west-2"
    )

    model_id = "anthropic.claude-3-haiku-20240307-v1:0"

    # Read and update the prompt with the job_id
    with open('prompt.txt', 'r') as file:
        prompt = file.read().replace("$$ID$$", 'job_id').replace("$$VALUE$$", job_id)
    
    # print(prompt)
    # Initialize the BedrockChat LLM using LangChain's BedrockChat class
    llm = BedrockChat(
        client=client,          # boto3 Bedrock client
        model_id=model_id,      # Model to use (e.g., Anthropic Claude v3)       # Max tokens for the response
    )

    # Set up the prompt template
    prompt_template = PromptTemplate(
        template=prompt,
        input_variables=["context", "question"]
    )
    # print(prompt_template)
    # Set up the retrieval chain with the language model and database retriever
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt_template},
        retriever=db.as_retriever(search_type="mmr", search_kwargs={'k': 9, 'lambda_mult': 0.5}),
    )


In [343]:
qstn = """
1.What is the percentage of jobs having quantity 1 during the period 2023.
2.What was the revenue by customer during the periods of Q1-2023.
3.What was the Revenue By License Program during the periods of Q1-2023.
4.What are the Top 5 Parts Shipped in period 2023-24?
5.What was the Revenue by Customer for Military during the period 2023
6.What was the Calibration Jobs By Customer during the period 2023  with Part number and description
7.what was the Revenue for tire cage Parts during the period 2023 to 2024 with part and description
8.What was the revenue by customer for market commercial during the period 2023?
9.What was the Revenue by Sales Partner during the periods of Q1-2023 with number of quotes
10.What was the Revenue by Value Stream during the period Q1-2023?
11.What was the Revenue by Value Stream during the period Q1-2022?
12.What is the percentage of jobs having quantity less than 6 during the period 2023
13.How many parts with descriptions containing 'tire' or 'cage' were produced in work orders and shipped in 2023?
"""

In [344]:
import re
import json
from QST import qstn

question = "How many parts with descriptions containing 'tire' or 'cage' were produced in work orders and shipped in 2023?"
job_id = "0fee44ac-bd4b-4217-a64d-a7dc33e00186"

chain1 = create_chain(db1, question, job_id)

In [345]:
%%time

import re
response = chain1.invoke(question)
result = response['result']
# print(result)

CPU times: user 87.3 ms, sys: 55.1 ms, total: 142 ms
Wall time: 4.56 s


In [346]:
import re

matches = re.findall(r'```cypher(.*?)```', response['result'], re.DOTALL)
for query in matches:
    print('%%oc\n',query.strip())

%%oc
 MATCH (wo:`Work Order`)-[:`produces`]-(p:`Part`)
WHERE wo.`ship_date` >= datetime('2023-01-01T00:00:00') AND wo.`ship_date` < datetime('2024-01-01T00:00:00')
  AND (toLower(p.`part_description`) CONTAINS 'tire' OR toLower(p.`part_description`) CONTAINS 'cage')
  AND wo.job_id = '0fee44ac-bd4b-4217-a64d-a7dc33e00186'
RETURN count(*) AS `Parts with 'tire' or 'cage' in description`;
